In [141]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [142]:
img_shape = 32
batch_size = 64

In [143]:
class Sampling(layers.Layer):

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [144]:
latent_dim = 64

In [145]:
encoder_inputs = keras.Input(shape=(32, 32, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 32, 32, 1)]  0           []                               
                                                                                                  
 conv2d_24 (Conv2D)             (None, 16, 16, 32)   320         ['input_27[0][0]']               
                                                                                                  
 conv2d_25 (Conv2D)             (None, 8, 8, 64)     18496       ['conv2d_24[0][0]']              
                                                                                                  
 flatten_12 (Flatten)           (None, 4096)         0           ['conv2d_25[0][0]']              
                                                                                            

In [146]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(16, activation="relu")(latent_inputs)
x = layers.Reshape((4, 4, 1))(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(128, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, 64)]              0         
                                                                 
 dense_27 (Dense)            (None, 16)                1040      
                                                                 
 reshape_14 (Reshape)        (None, 4, 4, 1)           0         
                                                                 
 conv2d_transpose_56 (Conv2D  (None, 8, 8, 32)         320       
 Transpose)                                                      
                                                                 
 conv2d_transpose_57 (Conv2D  (None, 16, 16, 64)       18496     
 Transpose)                                                      
                                                                 
 conv2d_transpose_58 (Conv2D  (None, 32, 32, 128)      7385

In [147]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


In [148]:
# (x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
# mnist_digits = np.concatenate([x_train, x_test], axis=0)
# mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

# vae = VAE(encoder, decoder)
# vae.compile(optimizer=keras.optimizers.Adam())
# vae.fit(mnist_digits, epochs=200, batch_size=32)


In [149]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=1, dtype=tf.uint8)
    image = tf.image.resize(image, [img_shape, img_shape])
    image = tf.cast(image, tf.float32)  # Convert the data type to float32
    image = image/255.0  # Normalize the images to [-1, 1]
    return image

list_ds = tf.data.Dataset.list_files('dataset/*.png')
dataset = list_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(tf.data.AUTOTUNE)


In [150]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01))
vae.fit(dataset, epochs=50, batch_size=batch_size)


Epoch 1/50
79/79 [==============================] - 7s 79ms/step - loss: 333.1769 - reconstruction_loss: 291.6717 - kl_loss: 0.0820
Epoch 2/50
79/79 [==============================] - 5s 58ms/step - loss: 275.6950 - reconstruction_loss: 274.6602 - kl_loss: 0.0050
Epoch 3/50
79/79 [==============================] - 5s 59ms/step - loss: 274.4748 - reconstruction_loss: 275.2353 - kl_loss: 3.9524e-04
Epoch 4/50
79/79 [==============================] - 5s 59ms/step - loss: 271.4402 - reconstruction_loss: 277.4440 - kl_loss: 8.5618e-05
Epoch 5/50
79/79 [==============================] - 5s 60ms/step - loss: 271.7606 - reconstruction_loss: 274.2249 - kl_loss: 8.5493e-05
Epoch 6/50
79/79 [==============================] - 5s 59ms/step - loss: 270.7309 - reconstruction_loss: 275.2268 - kl_loss: 3.8137e-05
Epoch 7/50
79/79 [==============================] - 5s 60ms/step - loss: 274.0872 - reconstruction_loss: 273.6979 - kl_loss: 5.2506e-05
Epoch 8/50
79/79 [==============================] - 5s 6

In [ ]:
latent_size = 64
num_samples = 10
latent_samples = np.random.normal(size=(num_samples, latent_size))

# Use the decoder to generate images from the latent space samples
generated_images = vae.decoder.predict(latent_samples)

# Visualize the generated images
for i in range(num_samples):
    plt.subplot(2, 5, i+1)
    plt.imshow(generated_images[i], cmap="gray")
    plt.axis("off")
plt.show()